# *import*

In [1]:
# from collections import OrderedDict
from typing import List, Tuple, Callable
import matplotlib.pyplot as plt
from datasets.utils.logging import enable_progress_bar
enable_progress_bar()
import time
import threading
import random
import os
import torch


import flwr
from flwr.client import Client, ClientApp
from flwr.common import ndarrays_to_parameters, Context, Metrics
from flwr.server import ServerApp, ServerConfig, ServerAppComponents

from flwr.simulation import run_simulation

from utils.model_CNN import Net
from utils.model_CNN import SVHNNet
from utils.model_CNN import ConvNet
from utils.train_test import test
from utils.loaddata import get_cached_datasets
from utils.others import get_parameters, set_parameters, evaluate_and_plot_confusion_matrix, weighted_average


from utils.history import history
from utils.client import FlowerClient
from utils.clientmanger import DynamicClientManager
from utils.HRFA_strategy import HRFA
from utils.other_strategy import AdaFedAdamStrategy, MyFedAvg
from utils.dynamic_controll import background_online_offline_simulator


DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Training on {DEVICE}")
print(f"Flower {flwr.__version__} / PyTorch {torch.__version__}")
# disable_progress_bar()

Training on cuda
Flower 1.17.0 / PyTorch 2.5.1+cu124


# **Load Data**

In [2]:
NUM_CLIENTS = 10
NUM_ROUNDS = 200
NUM_EPOCHS = 5
ATTACK_TYPE = "UPA"
DFL = False
STRATEGY = "HRFA"  # 可選 "FedAvg", "AdaFedAdam", "HRFA"

DATASET = "mnist" #"mnist", "cifar10", "svhn", "fashion_mnist"

# 攻擊者模式設定
ATTACK_MODE = "progressive"  # "fixed": 固定攻擊者數量, "progressive": 攻擊者漸漸變多
ATTACK_INCREASE_ROUNDS = NUM_ROUNDS*0.9  # 僅在 progressive 模式下使用：在前多少輪中逐漸增加攻擊者

#dynamic experiment
if DATASET == "cifar10":
    FRACTION = 0.6
    Q = 0.7
    ATTACK_NUMS = NUM_CLIENTS*0.28
    # ATTACK_NUMS = 0
    Net = ConvNet
elif DATASET == "fashion_mnist":
    FRACTION = 0.5
    Q = 0.8
    ATTACK_NUMS = NUM_CLIENTS*0.28
    # ATTACK_NUMS = 0
    Net = ConvNet
#static experiment
elif DATASET == "mnist":
    FRACTION = 1.0
    Q = 1.0
    ATTACK_NUMS = (NUM_CLIENTS//2)-1
    # ATTACK_NUMS = 0
    Net = Net
else :
    FRACTION = 1.0
    Q = 0.9
    ATTACK_NUMS = (NUM_CLIENTS//2)-1
    # ATTACK_NUMS = 0
    Net = SVHNNet


In [3]:
# trainloader, _, _ = get_cached_datasets(partition_id=0, dataset_name=DATASET, num_partitions=NUM_CLIENTS, q=Q)
# batch = next(iter(trainloader))
# images, labels = batch["img"], batch["label"]

# # Reshape and convert images to a NumPy array
# # matplotlib requires images with the shape (height, width, 3)
# images = images.permute(0, 2, 3, 1).numpy()

# # Denormalize
# images = images / 2 + 0.5

# # Create a figure and a grid of subplots
# fig, axs = plt.subplots(4, 8, figsize=(12, 6))

# # Loop over the images and plot them
# for i, ax in enumerate(axs.flat):
#     ax.imshow(images[i])
#     ax.set_title(trainloader.dataset.features["label"].int2str([labels[i]])[0])
    
#     ax.axis("off")

# # Show the plot
# fig.tight_layout()
# plt.show()

In [4]:
current_server_round = 0
new_round_event = threading.Event()

def get_current_round() -> int:
    return current_server_round

# **Client**

In [5]:
def client_fn(context: Context) -> Client:
    # Load model
    net = Net().to(DEVICE, memory_format=torch.channels_last)
    
	# Read the node_config to fetch data partition associated to this node
    partition_id = str(context.node_config["partition-id"])  # 強制轉換為字串
    num_partitions = context.node_config["num-partitions"]
    trainloader, valloader, _ = get_cached_datasets(partition_id, dataset_name=DATASET, num_partitions=NUM_CLIENTS, q=Q)

    if int(partition_id) < ATTACK_NUMS:
        return FlowerClient(partition_id, net, trainloader, valloader, ATTACK_TYPE, ATTACK_NUMS, ATTACK_MODE, ATTACK_INCREASE_ROUNDS).to_client()
    else:
        return FlowerClient(partition_id, net, trainloader, valloader).to_client()

# Create the ClientApp
client = ClientApp(client_fn=client_fn)

# **Server**

### setting

In [6]:
def server_evaluate(server_round, parameters, config):
    global current_server_round
    # 更新全域變數，讓背景執行緒知道目前是第幾個 round
    current_server_round = server_round
    """Evaluate the global model after each round (不再畫 confusion matrix)."""
    start_time = time.time()  # 記錄開始時間
    net = Net().to(DEVICE)
    set_parameters(net, parameters)

    # 加載測試集
    _, _, testloader = get_cached_datasets(0, dataset_name=DATASET, num_partitions=NUM_CLIENTS, q=Q)

    # 測試
    loss, accuracy = test(net, testloader)
    
    end_time = time.time()  # 記錄結束時間
    round_time = end_time - start_time  # 計算 round 時間
    
    history.add_loss_centralized(server_round, loss)
    history.add_metrics_centralized(server_round, {"accuracy": accuracy})

    # 只記錄最終模型，不畫 confusion matrix
    if server_round == NUM_ROUNDS:  # 最後一輪才返回模型
        evaluate_and_plot_confusion_matrix(net, testloader, DEVICE)
        return loss, {"accuracy": accuracy}
    new_round_event.set()
    return loss, {"accuracy": accuracy}


In [7]:
def fit_config(server_round: int):
    """Return training configuration dict for each round.

    Perform two rounds of training with one local epoch, increase to two local
    epochs afterwards.
    """
    config = {
        "server_round": server_round,  # The current round of federated learning
        # "local_epochs": 1 if server_round < 2 else NUM_EPOCHS,
        "local_epochs": NUM_EPOCHS,
        "train_mode": "lookahead", #no use
    }
    return config

In [8]:
params = get_parameters(Net())
param_count = sum(p.numel() for p in Net().parameters() if p.requires_grad)
print(f"Trainable Parameters: {param_count:,d}")

if STRATEGY == "AdaFedAdam":
    print("Using AdaFedAdam strategy")
    strategy = AdaFedAdamStrategy(
        fraction_fit=FRACTION,
        fraction_evaluate=FRACTION,
        
        min_fit_clients=int(NUM_CLIENTS * FRACTION),  # 確保是整數
        min_evaluate_clients=int(NUM_CLIENTS * FRACTION),  # 確保是整數
        min_available_clients=int(NUM_CLIENTS * FRACTION),  # 確保是整數

        initial_parameters=ndarrays_to_parameters(params),
        evaluate_fn=server_evaluate,
        evaluate_metrics_aggregation_fn=weighted_average,
        on_fit_config_fn=fit_config,
        on_evaluate_config_fn=fit_config,
        net=Net().to(DEVICE),
    )
elif STRATEGY == "FedAvg":
    print("Using FedAvg strategy")
    strategy = MyFedAvg(
        fraction_fit=FRACTION,
        fraction_evaluate=FRACTION,
        
        min_fit_clients=int(NUM_CLIENTS * FRACTION),  # 確保是整數
        min_evaluate_clients=int(NUM_CLIENTS * FRACTION),  # 確保是整數
        min_available_clients=int(NUM_CLIENTS * FRACTION),  # 確保是整數

        initial_parameters=ndarrays_to_parameters(params),
        evaluate_fn=server_evaluate,
        evaluate_metrics_aggregation_fn=weighted_average,
        on_fit_config_fn=fit_config,
        on_evaluate_config_fn=fit_config,
    )
else:
    print("Using HRFA strategy")
    strategy = HRFA(
        fraction_fit=FRACTION,  # Sample 100% of available clients for training
        fraction_evaluate=FRACTION,  # Sample 50% of available clients for evaluation
        
        min_fit_clients=2,  # Never sample less than 10 clients for training
        min_evaluate_clients=2,  # Never sample less than 5 clients for evaluation
        min_available_clients=2,  # Wait until all 10 clients are available
        
        initial_parameters=ndarrays_to_parameters(params),  # Pass initial model parameters
        evaluate_fn=server_evaluate,  # 設定 evaluate_fn
        evaluate_metrics_aggregation_fn=weighted_average,  # <-- pass the metric aggregation function
        on_fit_config_fn=fit_config,  # Pass the fit_config function
        on_evaluate_config_fn=fit_config,
        net=Net().to(DEVICE),
        # testloader = testloader,
    )


Trainable Parameters: 620,810
Using HRFA strategy


### Define Server

In [9]:
def server_fn(context: Context) -> ServerAppComponents:
    """Construct components that set the ServerApp behaviour."""
    global testloader

    # 設定 ServerConfig，如同你的程式碼
    config = ServerConfig(num_rounds=NUM_ROUNDS)

    # 建立動態管理器
    client_manager = DynamicClientManager()
        
    # 建立並啟動背景執行緒，模擬客戶端動態上/下線
    simulator_thread = threading.Thread(
        target=background_online_offline_simulator,
        args=(client_manager, get_current_round, new_round_event, 0.5, 0.5, DFL),  # interval=30秒, toggle_rate=0.3
        daemon=True  # 設 daemon=True 可以在主程式結束時自動退出
    )
    simulator_thread.start()

    return ServerAppComponents(
        strategy=strategy,
        config=config,
        client_manager=client_manager,
    )


# Create the ServerApp
server = ServerApp(server_fn=server_fn)

# *Run simulation*

### Run setting

In [10]:
# Specify the resources each of your clients need
# By default, each client will be allocated 1x CPU and 0x GPUs
backend_config = {"client_resources": {"num_cpus": 1, "num_gpus": 0.0}}

# When running on GPU, assign an entire GPU for each client
if DEVICE == "cuda":
    backend_config = {"client_resources": {"num_cpus": 2 , 
                                           "num_gpus": 0.2 #if (NUM_CLIENTS*FRACTION)>40 else 2/(NUM_CLIENTS*FRACTION)
                                          }}
    # Refer to our Flower framework documentation for more details about Flower simulations
    # and how to set up the `backend_config`

In [ ]:
# 讓 Flower 運行完整的 FL 訓練
start_time = time.time()  # 記錄開始時間

run_simulation(
    server_app=server,
    client_app=client,
    num_supernodes=NUM_CLIENTS,
    backend_config=backend_config,
)

end_time = time.time()  # 記錄結束時間
total_time = end_time - start_time  # 計算總時間

print(f"Total Training Time: {total_time:.2f} seconds")  # 顯示總時間

INFO :      Starting Flower ServerApp, config: num_rounds=200, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters


[Status] Client 17404053802592907449 已上線 ✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓
[Status] Client 7109502038303752640 已上線 ✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓
[Status] Client 16857353847917058018 已上線 ✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓
[Status] Client 12275152569355108357 已上線 ✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓
[Status] Client 14109915122520784357 已上線 ✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓
[Status] Client 14375932597445359172 已上線 ✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓
[Status] Client 1623556211981120490 已上線 ✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓
[Status] Client 14973552743668958058 已上線 ✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓
[Status] Client 16638695129468460329 已上線 ✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓
[Status] Client 12458978761388628182 已上線 ✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓
[Wed Jul  9 02:18:25 2025] 初始化聯邦數據集: mnist, 分區數=10, q=1.0


INFO :      initial parameters (loss, other metrics): 0.03617859191894531, {'accuracy': 0.1005}
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 10 clients (out of 10)


user online: 10 offline: 0
(ClientAppActor pid=666345) [Wed Jul  9 02:18:59 2025] 初始化聯邦數據集: mnist, 分區數=10, q=1.0


INFO :      aggregate_fit: received 10 results and 0 failures


...

[INFO] trained_partitions: {'7', '6', '8', '4', '2', '1', '5', '3', '0', '9'}
[INFO] shapley_values: {'16638695129468460329': np.float64(0.07193034324400986), '14109915122520784357': np.float64(0.7518564755322298), '17404053802592907449': 1.0, '14973552743668958058': 0.0, '14375932597445359172': np.float64(0.9681355988059839), '12458978761388628182': np.float64(0.5949094183158025), '7109502038303752640': np.float64(0.5380170872614576), '16857353847917058018': np.float64(0.29446440700756904), '1623556211981120490': np.float64(0.05276225748139003), '12275152569355108357': np.float64(0.287166191309459)}


INFO :      fit progress: (176, 0.004374663131311535, {'accuracy': 0.9086}, 32520.34707739501)
INFO :      configure_evaluate: strategy sampled 10 clients (out of 10)


user online: 10 offline: 0


INFO :      aggregate_evaluate: received 10 results and 0 failures
INFO :      
INFO :      [ROUND 177]
INFO :      configure_fit: strategy sampled 10 clients (out of 10)
INFO :      aggregate_fit: received 10 results and 0 failures


[INFO] trained_partitions: {'7', '6', '8', '4', '2', '1', '5', '3', '0', '9'}


### Loss 和 Accuracy 折線圖

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 提取每個 client（按 pid）的 loss 和 accuracy 數據
client_losses = {}  # {pid: [(round, loss), ...]}
client_accuracies = {}  # {pid: [(round, acc), ...]}

for round_num, pid, loss in history.losses_distributed:
    if pid not in client_losses:
        client_losses[pid] = []
    client_losses[pid].append((round_num, loss))

for round_num, pid, acc in history.metrics_distributed.get('accuracy', []):
    if pid not in client_accuracies:
        client_accuracies[pid] = []
    client_accuracies[pid].append((round_num, acc))

# 為圖表生成隨機顏色
colors = plt.cm.tab20(np.linspace(0, 1, max(len(client_losses), len(client_accuracies))))

# 繪製每個 client 的 Loss 圖
plt.figure(figsize=(10, 6))
for idx, (pid, losses) in enumerate(client_losses.items()):
    rounds = [r for r, _ in sorted(losses)]
    loss_values = [l for _, l in sorted(losses)]
    plt.plot(rounds, loss_values, marker='o', color=colors[idx], label=f'Partition {pid}', alpha=0.7)

# 繪製 centralized 和 distributed loss
rounds_loss = [t[0] for t in history.losses_centralized]
loss_values_centralized = [t[1] for t in history.losses_centralized]

distributed_loss_by_round = {}
for round_num, _, loss in history.losses_distributed:
    if round_num not in distributed_loss_by_round:
        distributed_loss_by_round[round_num] = []
    distributed_loss_by_round[round_num].append(loss)
rounds_loss_distributed = sorted(distributed_loss_by_round.keys())
loss_values_distributed = [np.mean(distributed_loss_by_round[r]) for r in rounds_loss_distributed]

plt.plot(rounds_loss, loss_values_centralized, marker='o', color='red', label='Centralized Loss', linewidth=2, linestyle='--')
# plt.plot(rounds_loss_distributed, loss_values_distributed, marker='o', color='orange', label='Distributed Loss', linewidth=2, linestyle='--')

plt.xlabel('Round')
plt.ylabel('Loss')
plt.title('Loss Over Rounds (Per Client by PID)')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.show()

# 繪製每個 client 的 Accuracy 圖
plt.figure(figsize=(10, 6))
for idx, (pid, accuracies) in enumerate(client_accuracies.items()):
    rounds = [r for r, _ in sorted(accuracies)]
    acc_values = [a for _, a in sorted(accuracies)]
    plt.plot(rounds, acc_values, marker='o', color=colors[idx], label=f'Partition {pid}', alpha=0.7)

# 繪製 centralized 和 distributed accuracy
acc_rounds = [t[0] for t in history.metrics_centralized.get('accuracy', [])]
acc_values_centralized = [t[1] for t in history.metrics_centralized.get('accuracy', [])]

distributed_acc_by_round = {} 
for round_num, _, acc in history.metrics_distributed.get('accuracy', []):
    if round_num not in distributed_acc_by_round:
        distributed_acc_by_round[round_num] = []
    distributed_acc_by_round[round_num].append(acc)
acc_rounds_md = sorted(distributed_acc_by_round.keys())
acc_values_md = [np.mean(distributed_acc_by_round[r]) for r in acc_rounds_md]

plt.plot(acc_rounds, acc_values_centralized, marker='o', color='blue', label='Centralized Accuracy', linewidth=2, linestyle='--')
# plt.plot(acc_rounds_md, acc_values_md, marker='o', color='green', label='Distributed Accuracy', linewidth=2, linestyle='--')

plt.xlabel('Round')
plt.ylabel('Accuracy')
plt.title('Accuracy Over Rounds (Per Client by PID)')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# ========= 訓練結束後繪圖 =========

# 假設 NUM_ROUNDS 為全局訓練輪數
rounds = list(range(1, NUM_ROUNDS + 1))
metrics_to_plot = ['reputation', 'short_term_rs', 'long_term_rs', 'similarity', 'shapley']

# 從策略中提取 metric_history
metric_history = strategy.metric_history

for metric in metrics_to_plot:
    plt.figure(figsize=(10, 6))
    for partition_id, metrics_dict in metric_history.items():
        # if partition_id == -1:
        #     continue
        values = metrics_dict[metric]
        # 若某 client 的紀錄少於 NUM_ROUNDS (代表部分 round 沒有更新)，則補上最後一次的值
        if len(values) < NUM_ROUNDS:
            last_val = values[-1] if values else 0.0
            values = values + [last_val] * (NUM_ROUNDS - len(values))
        plt.plot(rounds, values, marker='o', label=f'Partition {partition_id}')
    plt.xlabel('Round')
    plt.ylabel(metric)
    plt.title(f'{metric} over Rounds')
    plt.legend()
    plt.grid(True)
    plt.show()


In [ ]:
# history.metrics_centralized
best_epoch, best_accuracy = max(history.metrics_centralized['accuracy'], key=lambda x: x[1])

print(f"Best Epoch: {best_epoch}, Accuracy: {best_accuracy:.4f}")

In [ ]:
import numpy as np

# 1. 從 history.losses_distributed 中整理出：{ round_num: [loss_client1, loss_client2, ...], ... }
accuracies_by_round = {}
for round_num, pid, loss in history.metrics_distributed.get('accuracy', []):
    if round_num not in accuracies_by_round:
        accuracies_by_round[round_num] = []
    accuracies_by_round[round_num].append(loss)

# 2. 按 round 排序，計算每一輪的 client_loss 標準差（母體標準差 ddof=0）
rounds = sorted(accuracies_by_round.keys())
fairness_std_per_round = []
for r in rounds:
    client_accuracies_this_round = accuracies_by_round[r]
    if len(client_accuracies_this_round) > 1:
        std_val = float(np.std(client_accuracies_this_round, ddof=0))
    else:
        std_val = 0.0  # 如果只有一個 client 或無資料，就設為 0
    fairness_std_per_round.append(std_val)


avg_fairness_std = float(np.mean(fairness_std_per_round))
# print("500 輪公平性標準差平均值：", avg_fairness_std)
# fairness_std_per_round

# 確定只考慮前500輪（若總輪次少於500，則全部都考慮）
num_rounds = min(500, len(fairness_std_per_round))
subset = fairness_std_per_round[:num_rounds]

# 取最小值
min_std = min(subset)

print(f"前 {num_rounds} 輪中，公平性標準差的最小值為：{min_std}")

In [ ]:
# import torch
# import numpy as np
# from typing import List, Tuple, Dict
# from collections import OrderedDict
# from math import log, isnan
# from torch.utils.data import DataLoader
# import torch.nn as nn

# from flwr.client import NumPyClient

# from utils.train_test import test, lookahead_UPA_train, lookahead_TPA_train, train, local_evaluate
# from utils.others import get_parameters, set_parameters
# from utils.time import timed
# from utils.weights_utils import weights_substraction, norm

# DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# class FlowerClient(NumPyClient):
#     def __init__(self, partition_id, net, trainloader, valloader, attack_type: str = None):
#         self.partition_id = partition_id
#         self.net = net
#         self.trainloader = trainloader
#         self.valloader = valloader
#         self.device = DEVICE
#         self.optimizer = torch.optim.SGD(self.net.parameters(), lr=0.001, momentum=0.9)  # Adjusted lr
#         self.attack_type = attack_type

#     def get_parameters(self, config):
#         return [param.cpu().numpy() for param in self.net.state_dict().values()]

#     def get_parameters_dict(self, config):
#         """Helper to return parameters as an OrderedDict for weights_substraction."""
#         return OrderedDict([(name, param) for name, param in self.net.state_dict().items()])

#     def fit(self, parameters, config):
#         try:
#             if self.attack_type == "UPA":
#                 grad, current_loss = lookahead_UPA_train(self.net, self.trainloader, parameters, config, self.partition_id, verbose=False)
#             elif self.attack_type == "TPA":
#                 grad, current_loss = lookahead_TPA_train(self.net, self.trainloader, parameters, config, self.partition_id, verbose=False)
#             else:
#                 grad, current_loss = train(self.net, self.trainloader, parameters, config, self.partition_id, verbose=False)
            
#             # 計算確定性係數
#             try:
#                 certainty, weight = cal_certainty(self.net, parameters, grad, self.trainloader, self.device, self.optimizer)
#             except Exception as inner_e:
#                 import traceback
#                 print(f"客戶端 {self.partition_id} 計算更新時出錯: {inner_e}")
#                 print(traceback.format_exc())
#                 certainty = 1.0
#                 weight = len(self.trainloader.dataset)
            
#             torch.cuda.empty_cache() if torch.cuda.is_available() else None  # 清理 GPU 記憶體
#             # 確保返回正確的三元組格式
#             return self.get_parameters(config), weight, {"certainty": float(certainty), "partition_id": self.partition_id}
        
#         except Exception as e:
#             import traceback
#             print(f"客戶端 {self.partition_id} 訓練錯誤: {e}")
#             print(traceback.format_exc())
#             # 出錯時仍返回正確格式
#             return self.get_parameters(config), len(self.trainloader.dataset), {"certainty": 1.0, "partition_id": self.partition_id}

#     def evaluate(self, parameters, config):
#         try:
#             avg_loss, accuracy = local_evaluate(parameters, self.net, self.device, self.valloader, self.partition_id)
#             torch.cuda.empty_cache() if torch.cuda.is_available() else None  # 清理 GPU 記憶體
#             return float(avg_loss), len(self.valloader.dataset), {
#                 "client_accuracy": float(accuracy),
#                 "client_loss": float(avg_loss),
#                 "partition_id": self.partition_id
#             }
#         except Exception as e:
#             print(f"客戶端 {self.partition_id} 評估錯誤: {e}")
#             return 0.0, len(self.valloader.dataset), {
#                 "client_accuracy": 0.0,
#                 "partition_id": self.partition_id
#             }

# def cal_certainty(net, parameters, grad, trainloader, device, optimizer):
#     # 獲取全局參數和本地更新後參數
#     global_weights = OrderedDict()
#     for name, param in zip(net.state_dict().keys(), parameters):
#         global_weights[name] = torch.tensor(param, dtype=torch.float, device=device)
    
#     local_weights = OrderedDict()
#     for name, param in net.state_dict().items():
#         local_weights[name] = param.detach().clone().float()
    
#     # 計算更新 - 現在確保兩個參數都是 OrderedDict
#     update = weights_substraction(global_weights, local_weights)
    
#     # 計算範數
#     update_norm = norm(update) if update else 1.0
#     grad_norm = norm(grad) if grad else 1.0
    
#     # 避免除以零
#     if grad_norm > 0 and update_norm > 0:
#         norm_factor = update_norm / grad_norm
#         local_lr = optimizer.param_groups[0]['lr']
#         if norm_factor > 0 and local_lr > 0:
#             certainty = log(norm_factor / local_lr) + 1
#             # 確保確定性合理
#             certainty = 1.0 if isnan(certainty) else max(0.1, min(2.0, certainty))
#         else:
#             certainty = 1.0
#     else:
#         certainty = 1.0
    
#     # 樣本數作為權重
#     weight = len(trainloader.dataset)
    
#     return certainty, weight
